# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('users_behavior.csv')

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df['messages']=df['messages'].astype("int")
df['calls']=df['calls'].astype("int")

In [6]:
#проверка на дубликаты
df.duplicated().sum()

0

## Разбейте данные на выборки

In [7]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [8]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.4, 
                                                                            random_state=123
                                                                           ) 

Сперва отделил для обучающей выборки 60%

In [9]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


In [10]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test, 
                                                                            target_test, 
                                                                            test_size=0.5, 
                                                                            random_state=123
                                                                           ) 

In [11]:
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


In [12]:
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


Далее поделил пополам и получилось 3 выборки: 60-20-20

<ul><li>features_train, target_train - обучающая выборка
<li>features_valid, target_valid - валидационная выборка
<li>features_test, target_test - тестовая выборка

## Исследуйте модели

### DecisionTreeClassifier

In [13]:
best_accuracy = 0
best_depth = 0
for depth in range(1,12):
    model = DecisionTreeClassifier(max_depth=depth, random_state=123)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth, "; Точность:", best_accuracy)

Глубина дерева: 5 ; Точность: 0.8227060653188181


### RandomForestClassifier

In [14]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(5,51,5):
        for sample in range(1,6):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, 
                                           min_samples_leaf=sample, random_state=123)
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "; Количество деревьев:",
      best_est, "; мин. объектов в узле:", best_sample, "; Точность:", best_accuracy)

Глубина дерева: 9 ; Количество деревьев: 25 ; мин. объектов в узле: 4 ; Точность: 0.8398133748055988


### LogisticRegression

In [15]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Качество:", accuracy)

Качество: 0.7589424572317263


Самая простая и быстрая. но хромает точность

### Вывод
    

<ul><li> Наилучший результат точности у модели DecisionTreeClassifier - 0.8227060653188181
<li>Наилучший результат точности у модели RandomForestClassifier - 0.8398133748055988
<li>Наилучший результат точности у модели  LogisticRegression - 0.7589424572317263

Наилучший результат показала модель Случайнго леса с гиперпараметрами: Глубина дерева: 9 ; Количество деревьев: 25 ; мин. объектов в узле: 4, что дало на валидационной выборке точность - 0.8398133748055988.
Это отличный показатель и далее именно эти параметры я буду использовать для проверки на тестовой выборке.

## Проверьте модель на тестовой выборке

### DecisionTreeClassifier

In [16]:
model = DecisionTreeClassifier(max_depth=12, random_state=123)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Точность:", accuracy)

Точность: 0.7480559875583204


### RandomForestClassifier

In [17]:
model = RandomForestClassifier(max_depth=9, n_estimators=25, min_samples_leaf=4, random_state=123)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Точность:", accuracy)

Точность: 0.7931570762052877


### LogisticRegression

In [18]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Точность:", accuracy)

Точность: 0.7309486780715396


### Вывод

<ul><li>При проверке на тестовой выборке оказалась лучше модель Случайного леса с показателем точности - 0.7931570762052877
<li>А самая же худшая по точности оказалась модель логистической регрессии - 0.7309486780715396

## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность используем модель DummyClassifier

In [19]:
model = DummyClassifier(strategy='most_frequent', random_state=123)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print('Точность:', result)

Точность: 0.6936236391912908


Точность модели DummyClassifier ниже всех остальных, а значит наилучшая модель случайного леса адекватна и выполняет свою задачу.

## Общий вывод

<ul><li> Данные были заранее предобработаны.
<li> Было проведено исследование трёх моделей: DecisionTreeClassifier, RandomForestClassifier, LogisticRegression
<li> Самый лучший показатель точности был достигнут моделью RandomForestClassifier с гиперпараметрами: max_depth=9, n_estimators=25, min_samples_leaf=4
<li> Показатель точности лучшей модели  - 0.7931570762052877
<li> Также модели прошли проверку на адекватность путём сравнения с моделью DummyClassifier

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
